In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 24.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import numpy as np
import random
import re
import ast
import emoji
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/University/Research/SemEval 2025: Task 7

/content/drive/MyDrive/University/Research/SemEval 2025: Task 7


# loading dataframes

In [ ]:
def clean_text(text):
    # Remove emojis (using Unicode ranges and emoji module)
    text = emoji.replace_emoji(text, replace="")

    # Remove hashtags followed by a string (but not numbers)
    text = re.sub(r'#(?!\d)\w+', '', text)

    # Remove [USER] and [URL] placeholders
    text = re.sub(r'\[USER\]|\[URL\]', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Strip extra whitespace
    text = text.strip()

    return text

def apply_clean_text(data):

    if isinstance(data, tuple):
        return (clean_text(data[0]), clean_text(data[1]), data[2])

    elif isinstance(data, list):
        return [apply_clean_text(item) for item in data]

    return data

def sort_df(df, column, function, ascending = False):
    df_sorted = df.copy()
    df_sorted['text_length'] = df_sorted[column].apply(function)
    df_sorted = df_sorted.sort_values(by='text_length', ascending=ascending)
    return df_sorted

  Data Properties:

      posts_df:
          posts['text'] is either a tuple of 3 or an empty string
          posts['ocr'] is a list of tuples of 3 which is the ocr for each picture

      fact_checks_df:
          fact_checks_df['title'] is either a tuple of 3 or an empty string
          fact_checks_df['claim'] is always a tuple of 3
          fact_checks_df['claim'][2] is either a list of tuples of 2 (lang, probability) or a list of two elements [lang, probability]

      mapping_df:
          nothing especial

In [ ]:
fact_checks_df = pd.read_csv('./data/test data/fact_checks.csv').fillna('').set_index('fact_check_id')
posts_df = pd.read_csv('./data/test data/posts.csv').fillna('').set_index('post_id')
mapping_df = pd.read_csv('./data/original data/pairs.csv')

# clearing data: removing hashtags, emojis, etc.

In [ ]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s


for col in ['claim', 'title']:
    fact_checks_df[col] = fact_checks_df[col].apply(parse_col)
    fact_checks_df[col] = fact_checks_df[col].apply(apply_clean_text)


for col in ['ocr', 'text']:
    posts_df[col] = posts_df[col].apply(parse_col)
    posts_df[col] = posts_df[col].apply(apply_clean_text)

In [ ]:
fact_checks_df

,claim,instances,title
fact_check_id,,,
0,"(Are avocados good for you?, Are avocados good...","[(1525653998.0, 'https://metafact.io/factcheck...",
1,"(Can animals have headaches?, Can animals have...","[(1617955634.0, 'https://metafact.io/factcheck...",
2,"(Can we help prevent Alzheimer's with diet?, C...","[(1525653998.0, 'https://metafact.io/factcheck...",
3,(Do any benefits of alcohol outweigh the risks...,"[(1525653998.0, 'https://metafact.io/factcheck...",
4,"(Does acupuncture work for headaches?, Does ac...","[(1617955595.0, 'https://metafact.io/factcheck...",
...,...,...,...
372883,(Claudia Sheinbaum quiere cambiar la Constituc...,"[('2024-05-28 02:48:00+02', 'https://factual.a...","(Agence France-Presse ..., France Media Agency..."
372884,(Embaixador de Israel levanta cartaz comemoran...,"[('2024-05-27 22:34:00+02', 'https://checamos....","(Agence France-Presse ..., France Media Agency..."
372891,(The viral video shows African protesting at Q...,"[('2022-09-26 02:00:00+02', 'https://www.newsm...",(Fact Check: Viral Video Does NOT Show African...


In [ ]:
posts_df

,instances,ocr,verdicts,text
post_id,,,,
28094,"[('2023-08-24 13:31:03+00', 'tg')]",,,(Nowy BRICS będzie kontrolował 80 procent świa...
28095,"[('2023-06-29 16:42:00+00', 'tg')]",,,"("" Nie ma Zelenskiego, nie ma wojny"" - baner r..."
28101,"[('2022-09-18 19:22:27+00', 'tg')]",,,"(A student in Bremen, Germany found an online ..."
28102,"[('2023-02-07 00:05:36+00', 'tg')]",,,(W Amazonie można już nabyć książkę kucharską ...
28112,"[('2019-10-05 15:19:19+00', 'fb')]",[(Donald J. Trump We need a Civil War to figh...,['False information'],"(DONALD TRUMP IS TOTALLY INSANE., DONALD TRUMP..."
...,...,...,...,...
79490,"[('2024-01-16 22:38:06+00', 'ig')]",[(نبتة الماندرا كورا الصارخة من شافها اقرأ الك...,['False information.'],(هاد النبتة ظهرت ففيلم هاري بوتر وهي نبتة ذات ...
79492,"[('2024-03-20 10:26:45+00', 'ig')]",,['Partly false information. Reviewed by indepe...,(- A ministra do Planejamento já sugeriu 30% d...
79493,"[('2024-03-24 18:03:50+00', 'ig')]","[(Yo tenía, ponele, 12 años y me acuerdo que, ...",['Partly false information. Reviewed by indepe...,(EL DELIRIO DEL LARRETA ES TOTAL: Larreta acab...


In [ ]:
mapping_df

,post_id,fact_check_id
0,2228,33
1,2228,23568
2,2228,194577
3,2229,33
4,2229,23568
...,...,...
25738,14765,201987
25739,17076,201987
25740,18203,201987
25741,25641,201991


In [ ]:
posts_df[posts_df['text'].apply(lambda x: not isinstance(x, tuple) and len(x) > 0)]

#result => post['text'] is either a tuple of 3 or an empty string

,instances,ocr,verdicts,text
post_id,,,,


In [ ]:
fact_checks_df[fact_checks_df['claim'].apply(lambda x: not isinstance(x, tuple))]

#result => fact_checks_df['claim'] is always a tuple of 3

,claim,instances,title
fact_check_id,,,


In [ ]:
fact_checks_df[fact_checks_df['claim'].apply(lambda x: not isinstance(x[2][0], tuple))]

#result => fact_checks_df['claim'][2] is either a list of tuples of 3 or a list of two elements

,claim,instances,title
fact_check_id,,,
205751,(“In the first two job reports since I took th...,"[('2017-03-17 14:40:09+01', 'http://www.factch...","(Trump Twists Jobs Numbers, Trump Twists Jobs ..."
205752,"(""Muslim migrant beats up Dutch boy on crutche...","[('2017-11-30 15:16:22+01', 'http://www.factch...","(Trump Spreads False Anti-Muslim Tweet, Trump ..."
205753,"(Suggests global warming isn't necessarily ""a ...","[('2018-02-12 23:44:53+01', 'https://www.factc...","(Will Global Warming Benefit Civilization?, Wi..."
205754,"(""I’ve already authorized the construction of ...","[('2017-03-17 14:50:17+01', 'http://www.factch...","(Trump Twists Jobs Numbers, Trump Twists Jobs ..."
205755,(“George Soros Paid 3 Democrats Millions To Vo...,"[('2018-01-26 15:29:14+01', 'https://www.factc...","(Soros Didn't Pay for Shutdown Votes, Soros Di..."
...,...,...,...
372883,(Claudia Sheinbaum quiere cambiar la Constituc...,"[('2024-05-28 02:48:00+02', 'https://factual.a...","(Agence France-Presse ..., France Media Agency..."
372884,(Embaixador de Israel levanta cartaz comemoran...,"[('2024-05-27 22:34:00+02', 'https://checamos....","(Agence France-Presse ..., France Media Agency..."
372891,(The viral video shows African protesting at Q...,"[('2022-09-26 02:00:00+02', 'https://www.newsm...",(Fact Check: Viral Video Does NOT Show African...


In [ ]:
fact_checks_df.loc[205751].claim

('“In the first two job reports since I took the oath of office, we’ve already added nearly half a million new jobs.”',
 '“In the first two job reports since I took the oath of office, we’ve already added nearly half a million new jobs.”',
 ['eng', 1.0])

# preparing fact checks dataframe

In [ ]:
def extract_the_most_likely_language(claim):
  try:
    if isinstance(claim[2][0], tuple):
        return claim[2][0][0]
    else:
        return claim[2][0]
  except (IndexError, TypeError):
    return None

def generate_content(r):
    title = ""
    if isinstance(r['title'], tuple):
        title = r['title'][0]
    return f"title: ' {title} ' | claim: ' {r['claim'][0]} '"

In [ ]:
fact_checks_df['language'] = fact_checks_df['claim'].apply(extract_the_most_likely_language)
# fact_checks_df['content'] = fact_checks_df.apply(lambda r: generate_content(r), axis=1)
fact_checks_df['content'] = fact_checks_df.apply(lambda r: r['claim'][0], axis=1)

In [ ]:
fc_sorted = sort_df(fact_checks_df, 'content', len)
fc_sorted

,claim,instances,title,language,content,text_length
fact_check_id,,,,,,
9136,"(18 de novembro de 2016 Maurício Moraes, Étore...","[(1479427140.0, 'https://apublica.org/checagem...",(5 deslizes de Temer ao falar do Congresso no ...,por,"18 de novembro de 2016 Maurício Moraes, Étore ...",9647
100280,(Papa Francisco Condenado por Tráfico de Crian...,"[(1571270280.0, 'https://www.boatos.org/religi...",(Papa Francisco é condenado por tráfico de cri...,por,Papa Francisco Condenado por Tráfico de Crianç...,7610
9135,"(18 de novembro de 2016 Maurício Moraes, Étore...","[(1479427140.0, 'https://apublica.org/checagem...",(5 deslizes de Temer ao falar do Congresso no ...,por,"18 de novembro de 2016 Maurício Moraes, Étore ...",5582
36946,(Checamos frases de seis senadores no debate q...,"[(1456444740.0, 'https://apublica.org/checagem...","(6 blefes e meias-verdades sobre o pré-sal, 6 ...",por,Checamos frases de seis senadores no debate qu...,4638
146978,(Von PS - 18. Oktober 2018 7 3446 Share Immer ...,"[(1540545597.0, 'https://correctiv.org/fakten-...",(DSGVO – Namen an Klingelschildern sind zuläss...,deu,Von PS - 18. Oktober 2018 7 3446 Share Immer d...,4169
...,...,...,...,...,...,...
303302,"(, , [eng, 1.0])","[('2023-09-14 07:40:20+02', 'https://www.asian...","(നീണ്ട തലയും, മൂന്ന് വിരലുകളുള്ള കൈകളും, മെക്സ...",eng,,0
372696,"(, , [spa, 1.0])","[('2023-02-28 18:13:30+01', 'https://animalpol...","(Marcha del 26 de febrero fue a favor del INE,...",spa,,0
363657,"(, , [eng, 1.0])","[('2024-04-18 12:42:47+02', 'https://www.asian...","('ഇന്ത്യാ മുന്നണിക്ക് 326 സീറ്റുകള്‍, എന്‍ഡിഎ ...",eng,,0


In [ ]:
fc_sorted.head(3000)['content'].str.split().str.len().sum()

251740

# preparing posts dataframe

In [ ]:
# prompt: The ocr column of posts_df is a list of tuples of 3 element. their third element is a list of probable languages which is a tuple of the language name and its probability; so, the first first element of this language is the language of the ocr. check is there are row that the language of their ocrs is different (for example the language of the first ocr is english, while the language of the second ocr is french for one spicific post).

def check_ocr_language_consistency(ocr_list):
    if not ocr_list or not isinstance(ocr_list, list):
        return True  # Consider empty or invalid OCR as consistent

    first_language = None
    for ocr_tuple in ocr_list:
        try:
            current_language = ocr_tuple[2][0][0]
            if first_language is None:
                first_language = current_language
            elif current_language != first_language:
                return False  # Languages differ within the same post
        except (IndexError, TypeError):
            # Handle cases where OCR data might be incomplete or malformed
            return True #Consider this case as consistent

    return True  # All OCR languages are the same

# Find posts with inconsistent OCR languages
inconsistent_ocr_posts = posts_df[~posts_df['ocr'].apply(check_ocr_language_consistency)]

inconsistent_ocr_posts

,instances,ocr,verdicts,text
post_id,,,,


In [ ]:
def different_ocr_and_text_languages(row):
  try:
    ocr_lang = row['ocr'][0][2][0][0] if len(row['ocr']) > 0 else None
    text_lang = row['text'][2][0][0] if isinstance(row['text'], tuple) else None
    return ocr_lang != None and text_lang != None and ocr_lang != text_lang
  except (IndexError, TypeError):
    return False

# Display rows where OCR and text languages differ
multilingual_posts = posts_df[posts_df.apply(different_ocr_and_text_languages, axis=1)]
multilingual_posts

,instances,ocr,verdicts,text
post_id,,,,
28158,"[('2022-11-01 20:11:05+00', 'fb')]","[(InShot, InShot, [('rus', 1.0)])]",['False information'],"(Ratinho deixa recado para os arruaçeiros., Ra..."
28162,"[('2021-10-04 07:28:48+00', 'fb')]","[(ਇਹੀ ਆਣ ਤੂੰ ਸਜਾ, This is what you are punishe...",['This video may contain violent or graphic co...,(बरेली सिविल लाईन्स न्यूज़ पुलिस द्वारा चालान ...
28204,"[('2020-12-07 16:40:38+00', 'fb')]","[(World Health Organization, World Health Orga...",['False'],"(Podczas Rady Eugenicznej WHO w 2009 roku, Hen..."
28227,"[('2020-05-13 06:43:02+00', 'fb')]","[(THE DARK Msc.Snake, THE DARK Msc.Snake, [('e...",['Partly false information'],(نمر اثناء اصطيادة غزالة ولدت فاخذ النمر الصغي...
28228,"[('2022-11-02 03:09:06+00', 'fb')]",[(UCY RLS V/FOX NEWS 6:53 MT CANADA EXPANDING ...,['Missing context.'],"(INFO Kanada, Trudeau:wkrótce rozpocznie „Asys..."
...,...,...,...,...
79271,"[('2024-03-14 18:29:16+00', 'ig')]","[(actual CUESS boulanger, actual CUESS boulang...",['False information.'],(. لمن لا يعلم .. نادي ليل الفرنسي يوقع رسمياً...
79314,"[('2024-03-18 04:02:28+00', 'fb')]",[(lapatilla TUMBAN ESTATUA DE CHAVEZ Tik Tok ...,['Missing context.'],(DERRUMBAN estatua de CHÁVEZ. Los pueblos víct...
79335,"[('2024-03-06 12:16:57+00', 'fb')]","[(bradford, uk ل TikTok, bradford, uk ل TikTok...",['False information.'],(A cidade britânica de Bradford está completam...


In [ ]:
# prompt: build a data frame form posts_df by concatenating ocr texts of each row. the new data frame should have post_id as its index, a column named 'ocr' containing concatenated ocr, and a 'language' column containing the language of the first ocr. for the posts in inconsistent_posts instead of getting the first element of each ocr, get the second element and assign english as the language.

def build_dataframe(posts_df):
    inconsistent_ocr_posts_indexes = posts_df[~posts_df['ocr'].apply(check_ocr_language_consistency)].index
    multilingual_posts_indexes = posts_df[posts_df.apply(different_ocr_and_text_languages, axis=1)].index
    rows = []
    for index, row in posts_df.iterrows():
        ocr_texts = []
        text_text = ""
        eng_ocr_texts = []
        eng_text_text = ""
        language = None

        # get english translations
        for ocr_data in row['ocr']:
            try:
                eng_ocr_texts.append(ocr_data[1]) # Get the second element
            except (IndexError, TypeError):
                pass  # Skip if the data is malformed

        eng_text_text = row['text'][1] if isinstance(row['text'], tuple) else ""

        if index in inconsistent_ocr_posts_indexes or index in multilingual_posts_indexes:
            language = 'eng' # Assign English
            text_text = eng_text_text
            ocr_texts = eng_ocr_texts

        else:
            for ocr_data in row['ocr']:
                try:
                    ocr_texts.append(ocr_data[0])
                except (IndexError, TypeError):
                    pass # Skip if the data is malformed

            try:
              language = row['ocr'][0][2][0][0] if row['ocr'] else None
            except (IndexError, TypeError):
              language = None

            if isinstance(row['text'], tuple):
              text_text = row['text'][0]
              language = row['text'][2][0][0]

        rows.append({'post_id': index, 'ocr': ' '.join(ocr_texts), 'text': text_text, 'content': ' '.join(ocr_texts) + text_text, 'eng_content': ' '.join(eng_ocr_texts) + eng_text_text, 'language': language})

    new_df = pd.DataFrame(rows)
    new_df = new_df.set_index('post_id')
    return new_df

# Assuming posts_df and inconsistent_posts are defined as in your previous code
posts_df = build_dataframe(posts_df)
posts_df

,ocr,text,content,eng_content,language
post_id,,,,,
28094,,Nowy BRICS będzie kontrolował 80 procent świat...,Nowy BRICS będzie kontrolował 80 procent świat...,The new BRICS will control 80 percent of globa...,pol
28095,,""" Nie ma Zelenskiego, nie ma wojny"" - baner re...",""" Nie ma Zelenskiego, nie ma wojny"" - baner re...","&quot;No Zelensky, no war&quot; - New York ban...",pol
28101,,"A student in Bremen, Germany found an online s...","A student in Bremen, Germany found an online s...","A student in Bremen, Germany found an online s...",eng
28102,,W Amazonie można już nabyć książkę kucharską p...,W Amazonie można już nabyć książkę kucharską p...,Amazon now has a cookbook citing Klaus&#39; ne...,pol
28112,Donald J. Trump We need a Civil War to fight ...,DONALD TRUMP IS TOTALLY INSANE.,Donald J. Trump We need a Civil War to fight ...,Donald J. Trump We need a Civil War to fight ...,eng
...,...,...,...,...,...
79490,نبتة الماندرا كورا الصارخة من شافها اقرأ الكاب...,هاد النبتة ظهرت ففيلم هاري بوتر وهي نبتة ذات ا...,نبتة الماندرا كورا الصارخة من شافها اقرأ الكاب...,The screaming Mandarin Kura plant. Whoever see...,ara
79492,,- A ministra do Planejamento já sugeriu 30% de...,- A ministra do Planejamento já sugeriu 30% de...,- The Minister of Planning has already suggest...,por
79493,"Yo tenía, ponele, 12 años y me acuerdo que",EL DELIRIO DEL LARRETA ES TOTAL: Larreta acaba...,"Yo tenía, ponele, 12 años y me acuerdo queEL D...","I was, let&#39;s say, 12 years old and I remem...",spa


In [ ]:
set(posts_df['language'].to_list())

{'ara',
 'ben',
 'deu',
 'eng',
 'fra',
 'hin',
 'msa',
 'pol',
 'por',
 'spa',
 'tha',
 'tur',
 'urd',
 'zho'}

In [ ]:
posts_sorted = sort_df(posts_df, 'content', len)
posts_sorted

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
62898,24/11/2023 às 23h10min-Atualizada em 24/11/202...,Bill Gates ordena que o governo substitua alim...,24/11/2023 às 23h10min-Atualizada em 24/11/202...,24/11/2023 at 23:10-Updated on 24/11/2023 at 2...,por,13332
45378,52 MAM Frankenstadion,4. UPDATE Track bed pusher chronology of horro...,52 MAM Frankenstadion4. UPDATE Track bed pushe...,52 MAM Frankenstadion4. UPDATE Track bed pushe...,eng,11824
63817,ATTENTION: RCMP Evidence Enclosed Submitted Oc...,On December 31 2021 the original pcr test was ...,ATTENTION: RCMP Evidence Enclosed Submitted Oc...,ATTENTION: RCMP Evidence Enclosed Submitted Oc...,eng,10624
50106,STOWARZYSZENIE PRAWNIKÓW GŁOS WOLNOŚCI STOWARZ...,SKARGA W PRZEDMIOCIE REKLAMOWANIA SZCZEPIEŃ PR...,STOWARZYSZENIE PRAWNIKÓW GŁOS WOLNOŚCI STOWARZ...,LAWYERS&#39; ASSOCIATION VOICE OF FREEDOM LAWY...,pol,10539
43664,รับความตั้ง Overshoot u คนให้เห็นว่า 114 cm 18...,คนที่ชอบ swab บ่อยๆ อดทนอ่านสักนิดนะ อันนี้เค้...,รับความตั้ง Overshoot u คนให้เห็นว่า 114 cm 18...,Overshoot Day marks the 114th anniversary of t...,tha,10451
...,...,...,...,...,...,...
34191,,Don't try,Don't try,Don't try,eng,9
70683,,.,.,.,spa,1
72960,,,,,hin,0


In [ ]:
posts_sorted['content'].str.split().str.len().sum()

591956

# saving posts and fact checks dataframes

In [ ]:
posts_df

,ocr,text,content,eng_content,language
post_id,,,,,
28094,,Nowy BRICS będzie kontrolował 80 procent świat...,Nowy BRICS będzie kontrolował 80 procent świat...,The new BRICS will control 80 percent of globa...,pol
28095,,""" Nie ma Zelenskiego, nie ma wojny"" - baner re...",""" Nie ma Zelenskiego, nie ma wojny"" - baner re...","&quot;No Zelensky, no war&quot; - New York ban...",pol
28101,,"A student in Bremen, Germany found an online s...","A student in Bremen, Germany found an online s...","A student in Bremen, Germany found an online s...",eng
28102,,W Amazonie można już nabyć książkę kucharską p...,W Amazonie można już nabyć książkę kucharską p...,Amazon now has a cookbook citing Klaus&#39; ne...,pol
28112,Donald J. Trump We need a Civil War to fight ...,DONALD TRUMP IS TOTALLY INSANE.,Donald J. Trump We need a Civil War to fight ...,Donald J. Trump We need a Civil War to fight ...,eng
...,...,...,...,...,...
79490,نبتة الماندرا كورا الصارخة من شافها اقرأ الكاب...,هاد النبتة ظهرت ففيلم هاري بوتر وهي نبتة ذات ا...,نبتة الماندرا كورا الصارخة من شافها اقرأ الكاب...,The screaming Mandarin Kura plant. Whoever see...,ara
79492,,- A ministra do Planejamento já sugeriu 30% de...,- A ministra do Planejamento já sugeriu 30% de...,- The Minister of Planning has already suggest...,por
79493,"Yo tenía, ponele, 12 años y me acuerdo que",EL DELIRIO DEL LARRETA ES TOTAL: Larreta acaba...,"Yo tenía, ponele, 12 años y me acuerdo queEL D...","I was, let&#39;s say, 12 years old and I remem...",spa


In [ ]:
fact_checks_df

,claim,instances,title,language,content
fact_check_id,,,,,
0,"(Are avocados good for you?, Are avocados good...","[(1525653998.0, 'https://metafact.io/factcheck...",,eng,Are avocados good for you?
1,"(Can animals have headaches?, Can animals have...","[(1617955634.0, 'https://metafact.io/factcheck...",,eng,Can animals have headaches?
2,"(Can we help prevent Alzheimer's with diet?, C...","[(1525653998.0, 'https://metafact.io/factcheck...",,eng,Can we help prevent Alzheimer's with diet?
3,(Do any benefits of alcohol outweigh the risks...,"[(1525653998.0, 'https://metafact.io/factcheck...",,eng,Do any benefits of alcohol outweigh the risks?
4,"(Does acupuncture work for headaches?, Does ac...","[(1617955595.0, 'https://metafact.io/factcheck...",,eng,Does acupuncture work for headaches?
...,...,...,...,...,...
372883,(Claudia Sheinbaum quiere cambiar la Constituc...,"[('2024-05-28 02:48:00+02', 'https://factual.a...","(Agence France-Presse ..., France Media Agency...",spa,Claudia Sheinbaum quiere cambiar la Constituci...
372884,(Embaixador de Israel levanta cartaz comemoran...,"[('2024-05-27 22:34:00+02', 'https://checamos....","(Agence France-Presse ..., France Media Agency...",por,Embaixador de Israel levanta cartaz comemorand...
372891,(The viral video shows African protesting at Q...,"[('2022-09-26 02:00:00+02', 'https://www.newsm...",(Fact Check: Viral Video Does NOT Show African...,eng,The viral video shows African protesting at Qu...


In [ ]:
# posts_df.to_csv('./data/cleaned data/posts.csv')
# fact_checks_df.to_csv('./data/cleaned data/fact_checks.csv')

posts_df.to_csv('./data/cleaned data/posts_test.csv')
fact_checks_df.to_csv('./data/cleaned data/fact_checks_test.csv')

# Test Data

In [ ]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s


test_fact_checks_df = pd.read_csv('./data/test data/fact_checks.csv').fillna('').set_index('fact_check_id')
for col in ['claim', 'title']:
    test_fact_checks_df[col] = test_fact_checks_df[col].apply(parse_col)
    test_fact_checks_df[col] = test_fact_checks_df[col].apply(apply_clean_text)


test_posts_df = pd.read_csv('./data/test data/posts.csv').fillna('').set_index('post_id')
for col in ['ocr', 'text']:
    test_posts_df[col] = test_posts_df[col].apply(parse_col)
    test_posts_df[col] = test_posts_df[col].apply(apply_clean_text)

In [ ]:
test_fact_checks_df.iloc[10]['claim']

('"Dziś mamy ... nowy wielki atak nienawiści, bo te białe róże, które tam widać, to właśnie symbol nienawiści i głupoty, skrajnej głupoty i skrajnej nienawiści"',
 '"Today we have ... a new great attack of hatred, because those white roses you see there are a symbol of hatred and stupidity, extreme stupidity and extreme hatred"',
 [('pol', 1.0)])

In [ ]:
test_fact_checks_df = test_fact_checks_df.loc[~test_fact_checks_df.index.isin(fact_checks_df.index)]

In [ ]:
test_fact_checks_df['language'] = test_fact_checks_df['claim'].apply(extract_the_most_likely_language)
test_fact_checks_df['content'] = test_fact_checks_df.apply(lambda r: generate_content(r), axis=1)

In [ ]:
fc_sorted = sort_df(test_fact_checks_df, 'claim', lambda x: len(x[0]))
# fc_sorted = sort_df(test_fact_checks_df, 'content', len)
fc_sorted

,claim,instances,title,language,content,text_length
fact_check_id,,,,,,
246942,(Tradução de mensagem importante:\nDe uma enfe...,"[('2020-08-17 18:41:00+02', 'https://oglobo.gl...",(É que termômetro infravermelho cause dano à ...,p,title: ' É que termômetro infravermelho cause...,3159
253097,(Resposta da Churrascaria Fogo de Chão de NY s...,"[('2021-09-27 02:00:00+02', 'https://piaui.fol...","(Lupa, Lupa, [fil, 1.0])",p,title: ' Lupa ' | claim: ' Resposta da Churras...,2691
327685,(The fire of Notre Dame was the work of the Lu...,"[('2019-04-22 00:00:00+02', 'https://euvsdisin...",(Disinfo: The fire at the Notre Dame was a sat...,e,title: ' Disinfo: The fire at the Notre Dame w...,2487
320363,(The time will come soon when Euroliberal acti...,"[('2018-06-28 00:00:00+02', 'https://euvsdisin...",(Disinfo: Euroliberal activists will soon come...,e,title: ' Disinfo: Euroliberal activists will s...,2297
323087,(The trial in the Netherlands in the case of t...,"[('2021-06-22 00:00:00+02', 'https://euvsdisin...",(Disinfo: The US is hiding the truth from the ...,e,title: ' Disinfo: The US is hiding the truth f...,2289
...,...,...,...,...,...,...
363657,"(, , [eng, 1.0])","[('2024-04-18 12:42:47+02', 'https://www.asian...","('ഇന്ത്യാ മുന്നണിക്ക് 326 സീറ്റുകള്‍, എന്‍ഡിഎ ...",e,"title: ' 'ഇന്ത്യാ മുന്നണിക്ക് 326 സീറ്റുകള്‍, ...",0
304140,"(, , [eng, 1.0])","[('2023-09-19 11:14:11+02', 'https://www.asian...",(ഒരു എലിയെ പിടിക്കാന്‍ റെയില്‍വേ മുടക്കിയത് 41...,e,title: ' ഒരു എലിയെ പിടിക്കാന്‍ റെയില്‍വേ മുടക്...,0
372696,"(, , [spa, 1.0])","[('2023-02-28 18:13:30+01', 'https://animalpol...","(Marcha del 26 de febrero fue a favor del INE,...",s,title: ' Marcha del 26 de febrero fue a favor ...,0


In [ ]:
fc_sorted.iloc[700]

,325755
claim,(A muscular upper body is already an aggravati...
instances,"[('2021-06-02 00:00:00+02', 'https://euvsdisin..."
title,(Disinfo: Europeans organise an armed uprising...
language,eng
content,A muscular upper body is already an aggravatin...
text_length,902


In [ ]:
fc_sorted.iloc[8000].content

"title: ' Disinfo: Zelenskyy’s visit to Washington shows that Ukraine will be a neo-colony of the US ' | claim: ' The visit of Volodymyr Zelenskyy to Washington can be interpreted as Ukraine’s first step to becoming a neo-colony of the US. Zelenskyy’s request to obtain more US money has a complex geopolitical background, as it is some sort of bartering: money in exchange for autonomy. None of the money that Biden authorised are for free. '"

In [ ]:
fc_sorted.iloc[10000].text_length

382

In [ ]:
fc_sorted.iloc[10000].content

"title: ' Disinfo: There is no evidence of Russian involvement in the war in Donbas ' | claim: ' Ukraine is talking about some war. Ukraine is talking about something that in Europe, at least in Germany, cannot be understood. Everyone knows perfectly well the data of the BND (the German Foreign Intelligence Service) confirming that no one has ever seen Russian troops in Ukraine. '"

# Building positive samples

In [69]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s


fact_checks_df = pd.read_csv('./data/cleaned data/fact_checks.csv').fillna('').set_index('fact_check_id')

for col in ['claim', 'title']:
    fact_checks_df[col] = fact_checks_df[col].apply(parse_col)


posts_df = pd.read_csv('./data/cleaned data/posts.csv').fillna('').set_index('post_id')

mapping_df = pd.read_csv('./data/original data/pairs.csv')

with open('./data/original data/tasks.json', 'r') as file:
    tasks = json.load(file)

In [70]:
posts_df.drop(columns=[col for col in posts_df.columns if col not in ['eng_content', 'content']], inplace=True)

In [71]:
posts_df

,content,eng_content
post_id,,
0,! Dreister Impf-Fake von Markus Söder! Es ist ...,! Brazen vaccination fake by Markus Söder! It'...
1,!! WARNING !! A new thing circulating now. Peo...,!! WARNING !! A new thing circulating now. Peo...
2,"""Actually, he's a damn sight better than any o...","""Actually, he's a damn sight better than any o..."
3,"""Australia 50 MILLONES de dosis de ""vacuna"" re...","""Australia 50 MILLION doses of ""vacuna"" retira..."
4,"""Bienaventurados los perseguidos por mi causa ...","""Blessed are those persecuted by me cause ""The..."
...,...,...
28085,Se separó el presidente...la constitución de l...,The president separated... the constitution of...
28087,bruising runny ed 1 e Contents of the pack and...,bruising runny ed 1 e Contents of the pack and...
28089,"Robert Malone, inventeur de la technologie ARN...","Robert Malone, inventor of mRNA technology, st..."


In [72]:
fact_checks_df['claim_eng'] = fact_checks_df['claim'].apply(lambda x: x[1])
fact_checks_df['claim_org'] = fact_checks_df['claim'].apply(lambda x: x[0])

In [73]:
fact_checks_df.drop(columns=[col for col in fact_checks_df.columns if col not in ['claim_eng', 'claim_org']], inplace=True)

In [74]:
fact_checks_df

,claim_eng,claim_org
fact_check_id,,
0,Are avocados good for you?,Are avocados good for you?
1,Can animals have headaches?,Can animals have headaches?
2,Can we help prevent Alzheimer's with diet?,Can we help prevent Alzheimer's with diet?
3,Do any benefits of alcohol outweigh the risks?,Do any benefits of alcohol outweigh the risks?
4,Does acupuncture work for headaches?,Does acupuncture work for headaches?
...,...,...
205744,"In France, the military and civilian policemen...",في فرنسا ، يقرر رجال الشرطة العسكرية والمدنية ...
205745,This little beautiful girl was seen in Mangalo...,This little beautiful girl was seen in Mangalo...
205747,"Italians and foreigners, men and women, childr...",إيطاليين و أجانب رجال و نساء ، أطفال و عجزة ال...


In [77]:
merged = posts_df.merge(mapping_df, on='post_id', how='left')
merged = merged.merge(fact_checks_df, on='fact_check_id', how='left')

claims_eng_grouped = merged.groupby('post_id')['claim_eng'].apply(lambda x: x.dropna().tolist()).reset_index()
claims_org_grouped = merged.groupby('post_id')['claim_org'].apply(lambda x: x.dropna().tolist()).reset_index()

posts_df = posts_df.merge(claims_eng_grouped, on='post_id', how='left')
posts_df = posts_df.merge(claims_org_grouped, on='post_id', how='left')

posts_df['claim_eng'] = posts_df['claim_eng'].apply(lambda x: x if isinstance(x, list) else [])
posts_df['claim_org'] = posts_df['claim_org'].apply(lambda x: x if isinstance(x, list) else [])

In [82]:
posts_df

,post_id,content,eng_content,claim_eng,claim_org
0,0,! Dreister Impf-Fake von Markus Söder! Es ist ...,! Brazen vaccination fake by Markus Söder! It'...,[Markus Söder faked his vaccination.],[Markus Söder hat seine Impfung vorgetäuscht.]
1,1,!! WARNING !! A new thing circulating now. Peo...,!! WARNING !! A new thing circulating now. Peo...,[],[]
2,2,"""Actually, he's a damn sight better than any o...","""Actually, he's a damn sight better than any o...",[New Zealand opposition leader Judith Collins ...,[New Zealand opposition leader Judith Collins ...
3,3,"""Australia 50 MILLONES de dosis de ""vacuna"" re...","""Australia 50 MILLION doses of ""vacuna"" retira...",[The Australian government announces the withd...,[El gobierno de Australia anuncia la retirada ...
4,4,"""Bienaventurados los perseguidos por mi causa ...","""Blessed are those persecuted by me cause ""The...",[The photo shows nuns arrested for participati...,[La foto muestra a monjas detenidas por partic...
...,...,...,...,...,...
24426,28085,Se separó el presidente...la constitución de l...,The president separated... the constitution of...,"[In Uruguay, the president is required to be m...",[En Uruguay se exige que el presidente esté ca...
24427,28087,bruising runny ed 1 e Contents of the pack and...,bruising runny ed 1 e Contents of the pack and...,[Monkeypox is caused by a component of a coron...,[Affenpocken werden von einem Bestandteil eine...
24428,28089,"Robert Malone, inventeur de la technologie ARN...","Robert Malone, inventor of mRNA technology, st...",[Messenger RNA can damage the reproductive sys...,[L'ARN messager peut endommager le système rep...
24429,28090,Number of Covid-19 Deaths 4500 4000 3500 3000 ...,Number of Covid-19 Deaths 4500 4000 3500 3000 ...,[This data on deaths of vaccinees in England p...,[Estes dados de óbitos de vacinados na Inglate...


In [92]:
posts_df = posts_df.set_index('post_id')

In [93]:
posts_df

,content,eng_content,claim_eng,claim_org
post_id,,,,
0,! Dreister Impf-Fake von Markus Söder! Es ist ...,! Brazen vaccination fake by Markus Söder! It'...,[Markus Söder faked his vaccination.],[Markus Söder hat seine Impfung vorgetäuscht.]
1,!! WARNING !! A new thing circulating now. Peo...,!! WARNING !! A new thing circulating now. Peo...,[],[]
2,"""Actually, he's a damn sight better than any o...","""Actually, he's a damn sight better than any o...",[New Zealand opposition leader Judith Collins ...,[New Zealand opposition leader Judith Collins ...
3,"""Australia 50 MILLONES de dosis de ""vacuna"" re...","""Australia 50 MILLION doses of ""vacuna"" retira...",[The Australian government announces the withd...,[El gobierno de Australia anuncia la retirada ...
4,"""Bienaventurados los perseguidos por mi causa ...","""Blessed are those persecuted by me cause ""The...",[The photo shows nuns arrested for participati...,[La foto muestra a monjas detenidas por partic...
...,...,...,...,...
28085,Se separó el presidente...la constitución de l...,The president separated... the constitution of...,"[In Uruguay, the president is required to be m...",[En Uruguay se exige que el presidente esté ca...
28087,bruising runny ed 1 e Contents of the pack and...,bruising runny ed 1 e Contents of the pack and...,[Monkeypox is caused by a component of a coron...,[Affenpocken werden von einem Bestandteil eine...
28089,"Robert Malone, inventeur de la technologie ARN...","Robert Malone, inventor of mRNA technology, st...",[Messenger RNA can damage the reproductive sys...,[L'ARN messager peut endommager le système rep...


In [94]:
posts_df.to_csv('./data/cleaned data/posts_with_positive_samples.csv')